<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/DeepLearningNLP/seq2seq_model_11%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import pandas as pd


Dataset 준비하기
실제 실무에서와 같이 데이터 전처리 하는데에 시간이 꽤나 오래 걸립니다. 그러나 모델 작성은 그리 어렵지 않으니, 실습에 너무 큰 부담을 가지 마시길 바랍니다.

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip

In [ ]:
!unzip fra-eng.zip

In [ ]:
f = open('fra.txt','r')
lines = f.readlines()
print(lines[0])
print(lines[4])

eng_sents, fra_sents = [], []

# 한 문장을 영어와 불어를 분리해 따로 저장

for line in lines:
    eng, fra, _ = line.split('\t')
    eng_sents.append(eng)
    fra_sents.append(fra)

In [ ]:
eng_sents[0], fra_sents[0]

In [ ]:
print('데이터의 총 개 수', len(lines))
eng_sent_lengths = [len(eng_sent) for eng_sent in eng_sents]

In [ ]:
#EDA eng

eng_sents_series = pd.Series(eng_sents)
eng_sents_series.apply(len).describe()

In [ ]:
#EDA fra

fra_sents_series = pd.Series(fra_sents)
fra_sents_series.apply(len).describe()

In [ ]:
for ind, eng_sent in enumerate(eng_sents):
    eng_sents[ind] = eng_sent + ' '

for ind, fra_sent in enumerate(fra_sents):
    fra_sents[ind] = fra_sent + ' '

In [ ]:
# change char to number

eng_chars = set("".join(eng_sents))
print('# English chars : ',len(eng_chars))

# generate English char level dictionary
eng_char_dict = {}
for ind, eng_char in enumerate(eng_chars):
    eng_char_dict[eng_char] = ind


fra_chars = set("".join(fra_sents))
print('# English chars : ',len(fra_chars))

# generate Franch char level dictionary
fra_char_dict = {}
for ind, fra_char in enumerate(fra_chars):
    fra_char_dict[fra_char] = ind

In [ ]:
#change all english to number

eng_vectors = []

for eng_sent in eng_sents:
    eng_vectors.append([eng_char_dict[eng_char] for eng_char in eng_sent])

#change all franch to number

fra_vectors = []

for fra_sent in fra_sents:
    fra_vectors.append([fra_char_dict[fra_char] for fra_char in fra_sent])

In [ ]:
print(eng_sents[0], eng_vectors[0])
eng_char_dict['G'], eng_char_dict['o'], eng_char_dict['o'], eng_char_dict[' ']

print(fra_sents[0], fra_vectors[0])
fra_char_dict['V'], fra_char_dict['a'], fra_char_dict[' '], fra_char_dict['!']

In [ ]:
#change english vector to onehot vector
eng_input_onehots = [to_categorical(eng_vector, len(eng_chars)) for eng_vector in eng_vectors]

In [ ]:
fra_input_vectors = [fra_vector[:-1] for fra_vector in fra_vectors]
fra_output_vectors = [fra_vector[1:] for fra_vector in fra_vectors]

#change franch vector to onehot vector
fra_input_onehots = [to_categorical(fra_vector, len(fra_chars)) 
                    for fra_vector in fra_input_vectors]

fra_output_onehots = [to_categorical(fra_vector, len(fra_chars))
                       for fra_vector in fra_output_vectors]

In [ ]:
fra_vectors[0], fra_input_vectors[0], fra_output_vectors[0]

In [ ]:
train_xs = [eng_input_onehots, fra_input_onehots]
train_ys = fra_output_onehots


Build Encoder Model

In [ ]:
K.clear_session()
#encoder model
encoder_inputs = Input(shape=(None,len(eng_chars)))
latent_dim = 256
encoder = LSTM(latent_dim,return_state=True)
encoder_ouputs, encoder_h, encoder_c = encoder(encoder_inputs)
encoder_ouputs

Build Decoder Model

In [ ]:
decoder_inputs = Input(shape=(None, len(fra_chars)))

latent_dim = 256
decoder = LSTM(latent_dim,return_state=True, return_sequences=True)
decoder_ouputs, _, _ = decoder(decoder_inputs, initial_state=[encoder_h, encoder_c])
decoder_ouputs = Dense(units=len(fra_chars), activation='softmax')(decoder_ouputs)

Model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_ouputs)

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tqdm import tqdm

epochs = 1

for epoch in range(epochs):
    epoch_acc = []
    epoch_loss = []
    for ind in tqdm(range(len(train_xs[0]))):
        xs_0 = np.array(train_xs[0][ind: ind+1])
        xs_1 = np.array(train_xs[1][ind: ind+1])
        ys = np.array(train_ys[ind: ind+1])

        hist = model.fit([xs_0,xs_1],ys, batch_size=1, epochs=1, verbose = False)
        epoch_acc.append(hist.history['accuracy'])
        epoch_loss.append(hist.history['loss'])
    print('acc', np.mean(epoch_acc))
    print('loss', np.mean(epoch_loss))